In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split
from joblib import dump

Загрузим обработанные данные и поделим на тестовую и обучающую выборки.

In [2]:
train_df = pd.read_csv('train_modified.csv')
print(train_df.head())

        329      330       331       332      333      334       335  \
0  0.113296  0.13582  0.889617  0.002498  0.01252  0.00426  0.055041   
1  0.113296  0.13582  0.889617  0.002498  0.01252  0.00426  0.055041   
2  0.113296  0.13582  0.889617  0.002498  0.01252  0.00426  0.055041   
3  0.113296  0.13582  0.889617  0.002498  0.01252  0.00426  0.055041   
4  0.113296  0.13582  0.889617  0.002498  0.01252  0.00426  0.055041   

        336       337       338  ...       353       354      355       356  \
0  0.118789  0.034067  0.086975  ...  0.006501  0.002727  0.04035  0.043507   
1  0.118789  0.034067  0.086975  ...  0.006501  0.002727  0.04035  0.043507   
2  0.118789  0.034067  0.086975  ...  0.006501  0.002727  0.04035  0.043507   
3  0.118789  0.034067  0.086975  ...  0.006501  0.002727  0.04035  0.043507   
4  0.118789  0.034067  0.086975  ...  0.006501  0.002727  0.04035  0.043507   

        357       358  id       lat       lon     score  
0  0.103746  0.091576   0  0.26358

In [3]:
X = train_df.drop(labels=['score', 'id'], axis=1)
y= train_df['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Мой выбор был между моделями LightGBM и XGBoost в силу их быстроты и высокой производительности, а также устойчивости к переобучению. Обе модели показали неплохие результаты, однако XGBoost показала себя лучше. 

Настроим параметры с помощью поиска по сетке и обучим модель.

In [4]:
xgb_model = XGBRegressor(random_state=42)

xgb_params = {
    'max_depth': range(10, 20, 2),
    'learning_rate': [0.01, 0.05, 0.03],
    'subsample': [0.7, 0.8, 0.9, 1], 
}

grid_search = GridSearchCV(xgb_model, xgb_params,cv=5, n_jobs=4, 
                             verbose=True, scoring='neg_mean_absolute_error')

grid_search.fit(X_train, y_train)
print("Лучшие параметры:", grid_search.best_params_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Лучшие параметры: {'learning_rate': 0.05, 'max_depth': 18, 'subsample': 1}


In [5]:
model = XGBRegressor(learning_rate=0.05, max_depth=18, subsample=0.9, n_estimators=500, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.041404703596981125


Сохраним модель для дальнейшего использования.

In [6]:
dump(model, 'model_geo.joblib')

['model_geo.joblib']